# Training and hosting SageMaker Models using the Apache MXNet Module API

The **SageMaker Python SDK** makes it easy to train and deploy MXNet models. In this example, we train a simple neural network using the Apache MXNet [Module API](https://mxnet.incubator.apache.org/api/python/module.html) and the MNIST dataset. The MNIST dataset is widely used for handwritten digit classification, and consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). The task at hand is to train a model using the 60,000 training images and subsequently test its classification accuracy on the 10,000 test images.

### Setup

First we need to define a few variables that will be needed later in the example.

In [128]:
from sagemaker import get_execution_role
import json
import boto3
import time
import os
import time
import tarfile
cf = boto3.client('cloudformation')
s3 = boto3.client('s3')
sns = boto3.client('sns')
step = boto3.client('stepfunctions')
sagemaker = boto3.client('sagemaker-runtime')




with open('../config.json') as json_file:  
    config = json.load(json_file)
StackName=config["StackName"]

result=cf.describe_stacks(
    StackName=StackName
)
outputs={}
for output in result['Stacks'][0]['Outputs']:
    outputs[output['OutputKey']]=output['OutputValue']

with tarfile.open("script.tar.gz", "w:gz") as tar:
    tar.add(os.getcwd(),arcname="")

s3.upload_file("script.tar.gz",outputs["CodeBucket"],"script.tar.gz")
#IAM execution role that gives SageMaker access to resources in your AWS account.
#We can use the SageMaker Python SDK to get the role from our notebook environment. 
role = get_execution_role()

In [112]:
!ls 


input.html  mnist.py  mxnet_mnist.ipynb  script  script.tar.gz	tmp


In [124]:
result=cf.describe_stacks(
    StackName=StackName
)

params={}
for param in result['Stacks'][0]['Parameters']:
    params[param['ParameterKey']]=param['ParameterValue']
    
params["ConfigPresetType"]="MXNET"
params["HyperParameters"]=json.dumps({'learning_rate': 0.1})
params["TrainEntryPoint"]="mnist.py"
params["TrainSourceFile"]="s3://{}/script.tar.gz".format(outputs["CodeBucket"])
params["HostEntryPoint"]="mnist.py"
params["HostSourceFile"]="s3://{}/script.tar.gz".format(outputs["CodeBucket"])

Parameters=[ {"ParameterKey":param, "ParameterValue":params[param]} for param in params]
cf.update_stack(
    StackName=StackName,
    UsePreviousTemplate=True,
    Parameters=Parameters,
    Capabilities=['CAPABILITY_NAMED_IAM']
)
waiter = cf.get_waiter('stack_update_complete')
waiter.wait(
    StackName=StackName,
    WaiterConfig={
        'Delay': 10,
        'MaxAttempts': 123
    }
)


ClientError: An error occurred (ValidationError) when calling the UpdateStack operation: No updates are to be performed.

In [125]:
result=sns.publish(
    TopicArn=outputs['LaunchTopic'],
    Message="{}" #message is not important, just publishing to topic starts build
)
print(result)

{'MessageId': '18906df8-765c-5862-8bdb-3e6a0861015d', 'ResponseMetadata': {'RequestId': '9704fa2d-e4cc-5f43-957b-a9bf9af1d67c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9704fa2d-e4cc-5f43-957b-a9bf9af1d67c', 'content-type': 'text/xml', 'content-length': '294', 'date': 'Thu, 07 Jun 2018 00:27:40 GMT'}, 'RetryAttempts': 0}}


In [117]:
region = boto3.Session().region_name
print(region)

train_data_location = 's3://sagemaker-sample-data-{}/mxnet/mnist/train'.format(region)
test_data_location = 's3://sagemaker-sample-data-{}/mxnet/mnist/test'.format(region)

s3.


SyntaxError: invalid syntax (<ipython-input-117-4317c25a0a5a>, line 7)

In [123]:
%%time 
#list all executions for our StateMachine to get our current running one
result=step.list_executions(
    stateMachineArn=outputs['StateMachine'],
    statusFilter="RUNNING"
)['executions']

if len(result) > 0:
    response = step.describe_execution(
        executionArn=result[0]['executionArn']
    )
    status=response['status']
    print(status,response['name'])
    #poll status till execution finishes
    while status == "RUNNING":
        print('.',end="")
        sleep(5)
        status=step.describe_execution(executionArn=result[0]['executionArn'])['status']
    print()
    print(status)
else:
    print("no running tasks")


no running tasks
CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 134 ms


### The training script

The ``mnist.py`` script provides all the code we need for training and hosting a SageMaker model. The script we will use is adaptated from Apache MXNet [MNIST tutorial (https://mxnet.incubator.apache.org/tutorials/python/mnist.html).

In [ ]:
!cat mnist.py

### SageMaker's MXNet estimator class

The SageMaker ```MXNet``` estimator allows us to run single machine or distributed training in SageMaker, using CPU or GPU-based instances.

When we create the estimator, we pass in the filename of our training script, the name of our IAM execution role, and the S3 locations we defined in the setup section. We also provide a few other parameters. ``train_instance_count`` and ``train_instance_type`` determine the number and type of SageMaker instances that will be used for the training job. The ``hyperparameters`` parameter is a ``dict`` of values that will be passed to your training script -- you can see how to access these values in the ``mnist.py`` script above.

For this example, we will choose one ``ml.m4.xlarge`` instance.

In [ ]:
mnist_estimator = MXNet(entry_point='mnist.py',
                        role=role,
                        output_path=model_artifacts_location,
                        code_location=custom_code_upload_location,
                        train_instance_count=1, 
                        train_instance_type='ml.m4.xlarge',
                        hyperparameters={'learning_rate': 0.1})

### Running the Training Job

After we've constructed our MXNet object, we can fit it using data stored in S3. Below we run SageMaker training on two input channels: **train** and **test**.

During training, SageMaker makes this data stored in S3 available in the local filesystem where the mnist script is running. The ```mnist.py``` script simply loads the train and test data from disk.

In [ ]:
%%time
import boto3

region = boto3.Session().region_name
train_data_location = 's3://sagemaker-sample-data-{}/mxnet/mnist/train'.format(region)
test_data_location = 's3://sagemaker-sample-data-{}/mxnet/mnist/test'.format(region)

mnist_estimator.fit({'train': train_data_location, 'test': test_data_location})

### Creating an inference Endpoint

After training, we use the ``MXNet estimator`` object to build and deploy an ``MXNetPredictor``. This creates a Sagemaker **Endpoint** -- a hosted prediction service that we can use to perform inference. 

The arguments to the ``deploy`` function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances. Here we will deploy the model to a single ``ml.m4.xlarge`` instance.

In [ ]:
%%time

predictor = mnist_estimator.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

The request handling behavior of the Endpoint is determined by the ``mnist.py`` script. In this case, the script doesn't include any request handling functions, so the Endpoint will use the default handlers provided by SageMaker. These default handlers allow us to perform inference on input data encoded as a multi-dimensional JSON array.

### Making an inference request

Now that our Endpoint is deployed and we have a ``predictor`` object, we can use it to classify handwritten digits.

To see inference in action, draw a digit in the image box below. The pixel data from your drawing will be loaded into a ``data`` variable in this notebook. 

*Note: after drawing the image, you'll need to move to the next notebook cell.*

In [142]:
from IPython.display import HTML
HTML(open("input.html").read())

Now we can use the ``predictor`` object to classify the handwritten digit:

In [145]:
result=sagemaker.invoke_endpoint(
    EndpointName=outputs['SageMakerEndpoint'],
    Body=json.dumps(data),    
    ContentType="application/json",
    Accept="application/json"
)

response =json.loads(result['Body'].read().decode('utf-8'))
labeled_predictions = list(zip(range(10), response[0]))
print('Labeled predictions: ')
print(labeled_predictions)

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print('Most likely answer: {}'.format(labeled_predictions[0]))

Labeled predictions: 
[(0, 0.11481616646051407), (1, 1.252840206689143e-06), (2, 0.8315169811248779), (3, 0.004089549649506807), (4, 8.858233970562152e-14), (5, 0.040537502616643906), (6, 1.7384726334057632e-06), (7, 0.00038907164707779884), (8, 0.0086472537368536), (9, 4.3090787471555814e-07)]
Most likely answer: (2, 0.8315169811248779)


# (Optional) Delete the Endpoint

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it.

In [ ]:
print("Endpoint name: " + predictor.endpoint)

In [ ]:
import sagemaker

sagemaker.Session().delete_endpoint(predictor.endpoint)